In [ ]:
import math
import random

import drawsvg as draw
from drawsvg import Drawing
from hyperbolic import euclid, util
from hyperbolic.poincare import *

In [ ]:
def hyp_poly_edge_construct(p, q):
    pi, pi2 = math.pi, math.pi*2
    th = pi2/q
    phi = pi2/p
    ang1 = pi-phi/2-th/2-pi/2
    ang2 = th/2 + pi/2
    a = math.sin(ang2)/math.sin(ang1)
    b = math.sin(phi/2)/math.sin(ang1)
    r_p = math.sqrt(1/(a**2-b**2))
    r_c = a*r_p
    r_from_c = b*r_p
    #return r_c, r_from_c
    t1 = pi - math.asin(r_c / (r_from_c / math.sin(phi/2)))
    t2 = pi - t1 - phi/2
    r = math.sin(t2) * (r_from_c / math.sin(phi/2))
    return r

In [ ]:
p = 5
corner_deg = 90
q = 360 / corner_deg
skip = 1.0
r = hyp_poly_edge_construct(p, q)
p_list = [
    Point.from_polar_euclid(r, deg=-skip*i*360/p)
    for i in range(p)
]
print(p_list)

n = len(p_list)
e_list = [Line.from_points(*p_list[i], *p_list[(i+1)%n]) for i in range(n)]
poly = Polygon(e_list, join=True)

d = Drawing(2.1, 2.1, origin='center')
d.draw(euclid.Circle(0, 0, 1), fill='silver')
for edge in e_list:
    d.draw(edge, hwidth=0.05, fill='blue')
# d.draw(poly, hwidth=(0,-0.15), fill='green')
# d.draw(poly, fill='black', opacity=0.3)
# for p in p_list:
#     d.draw(p, radius=0.05, stroke_width=0.01, stroke='#ccccff',
#               fill='none', opacity=0.6)

d.set_render_size(w=400)
d.save_svg('images/polyRegular.svg')
d

In [ ]:
def construct_poly_vertices(p, q, deg_offset=0, skip=1):
    r = hyp_poly_edge_construct(p, q)
    return [
        Point.from_polar_euclid(r, deg=-skip * i * 360 / p + deg_offset)
        for i in range(p)
    ]

def construct_poly_from_points(pt_list):
    p = len(p_list)
    e_list = [Line.from_points(*pt_list[i], *pt_list[(i + 1) % p]) for i in range(p)]
    return Polygon(e_list, join=True)

def draw_poly(d, poly, color="#ffffff"):
    d.draw(poly, fill=color, opacity=0.25)
    d.draw(poly, hwidth=(0.0,0.05), fill='blue')
    
def draw_points(d, p_list):
    for pt in p_list:
        d.draw(pt, hradius=0.15, hwidth=0.02,
                   fill='white', opacity=0.9)

def ring(p, q, i0, i1, polygon_center_src):
    # construct a new poly centered around the origin (pts_center_src could be anywhere)
    pts_base = construct_poly_vertices(p, q)
    # TODO(lucasw) not sure what this does, why it only uses the first two points
    t0 = pts_base[0]
    t1 = pts_base[1]
    trans_to_origin = Transform.shift_origin(t0, t1)
    # print(f"trans_to_origin: {t0} {t1} -> {trans_to_origin}")
    if False:
        px, py = trans_to_origin.apply_to_tuple(pts_base[-1])
         # print('Inner angle:', math.degrees(math.atan2(py,px)))

    polygons = []
    # TODO(lucasw) need a datastructure that has a set of nodes with five neighbors each
    # and be able to track if a node has already been constructed in a given position
    
    # iterate through each edge of the polygon, and add a neighbor polygon
    for i in range(i0, i1):
        t0 = polygon_center_src.vertices[(i + 1) % p]
        t1 = polygon_center_src.vertices[i]
        # this seems like it would only be half the needed translation
        # it's the location of one edge of the center source polygon
        trans_to_side = Transform.translation(t0, t1)
        # print(f"{i} t: {tx} {ty} -> {trans_to_side}")
        transform = Transform.merge(trans_to_origin, trans_to_side)
        transformed_points = transform.apply_to_list(pts_base)
        transformed_polygon = Polygon.from_vertices(transformed_points)
        polygons.append(transformed_polygon)
   
    for polygon in polygons:
        draw_poly(d, polygon)
        
    # draw_points(d, p_list)
    # for pt_list2 in pt_list_list:
    #     draw_points(d, pt_list2)

    return polygons
    
# for ind in range(50):
ind = 25
if True:
    d = Drawing(2.1, 2.1, origin='center')
    d.draw(euclid.Circle(0, 0, 1), fill='#f9f9f9')

    p = 5
    q = 4
    # print('Inner angle:', 360/q)

    # the central polygon
    pt_list = construct_poly_vertices(p, q)
    rot_trans = Transform.rotation(deg=270)
    # offset = Transform.translation(Point(-0.5 + ind * 0.025, -0.0))
    # offset = Transform.translation(Point(-0.0, -0.5 + ind * 0.025))
    offset = Transform.translation(Point(0.25, 0.0))
    trans = Transform.merge(offset, rot_trans)
    pt_list = trans.apply_to_list(pt_list)
    poly = Polygon.from_vertices(pt_list)
    draw_poly(d, poly)

    polygons = ring(p, q, i0=0, i1=p, polygon_center_src=poly)

    # TODO(lucasw) need to avoid drawing the same shape twice
    if False:
        for polygon in polygons:
            polygons = ring(p, q, 1, 4, polygon)
            for polygon in polygons:
                polygons = ring(p, q, 0, 4, polygon)
                for polygon in polygons:
                    polygons = ring(p, q, 0, 5, polygon)
                    for polygon in polygons:
                        polygons = ring(p, q, 0, 5, polygon)

    # d.set_render_size(w=1080)
    d.set_render_size(w=400)
    name = f'images/poly_tile_{p}_{q}_{ind:05d}'
    d.save_png(name + ".png")
    # d.save_svg(name + ".svg")
d

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from hyperbolic.euclid import Arc, Line

def get_poly_xy(poly):
    xs = []
    ys = []
    for edge in poly.edges:
        arc = edge.proj_shape
        if isinstance(arc, Arc):
            num = 8
            for i in range(num):  # for deg in np.arange(arc.start_deg, arc.end_deg, 1.0):
                # Arc ought to have a convenience function for this
                # clockwise
                if arc.cw:
                    d0 = arc.start_deg
                    d1 = arc.end_deg
                    if d1 < d0:
                        d1 += 360
                else:
                    d0 = arc.start_deg
                    d1 = arc.end_deg
                    if d1 > d0:
                        d1 -= 360
                    
                deg = d0 + i * (d1 - d0) / num
                x = arc.cx + arc.r * math.cos(math.radians(deg))
                y = arc.cy + arc.r * math.sin(math.radians(deg))
                xs.append(x)
                ys.append(y)
        elif isinstance(arc, Line):
            xs.append(arc.x1)
            ys.append(arc.y1)
        else:
            print(type(arc))
        # pt0 = edge.start_point()
        # xs.append(pt0.x)
        # ys.append(pt0.y)
        # pt0 = edge.midpoint_euclid()
        # xs.append(pt0.x)
        # ys.append(pt0.y)

    xs.append(xs[0])
    ys.append(ys[0])
    
    return xs, ys

# print(pt_list)

xs, ys = get_poly_xy(poly)

fig, ax = plt.subplots()

for pt in pt_list:
    ax.plot(pt.x, pt.y, 'o')
    

# print(xs)
# print(ys)
    
ax.plot(xs, ys, '.')
ax.axis('equal')

In [ ]:
print(poly.vertices)

fig, ax = plt.subplots()
    
for poly in polygons:
    xs = []
    ys = []
    for vt in poly.vertices:
        xs.append(vt.x)
        ys.append(vt.y)
        
    xs.append(xs[0])
    ys.append(ys[0])
    
    ax.plot(xs, ys)
    ax.axis('equal')

In [ ]:
class Node:
    """construct a tree of polygon nodes
    avoid creating the same 
    """
    def __init__(self, name, depth, max_depth=1, parent=None, parent_side=None, p=5, q=4):
        self.name = name
        
        self.neighbors = {}
        for i in range(p):
            self.neighbors[i] = None
        
        self.parent = parent
        self.neighbors[0] = parent
        
        # TODO(lucasw) this is the same for every polygon
        vertices = construct_poly_vertices(p, q)
        # use the first edge as the shift origin
        t0 = vertices[0]
        t1 = vertices[1]
        transform = Transform.shift_origin(t0, t1)
        
        if self.parent is not None:
            t0 = parent.polygon.vertices[(parent_side + 1) % p]
            t1 = parent.polygon.vertices[parent_side]
            trans_to_side = Transform.translation(t0, t1)
            transform = Transform.merge(transform, trans_to_side)
        transformed_vertices = transform.apply_to_list(vertices)
        self.polygon = Polygon.from_vertices(transformed_vertices)
        
        self.children = {}
        if depth >= max_depth:
            return
        
        for i in range(p):
            if self.neighbors[i] is not None:
                continue
            node = Node(name=i, depth=depth + 1, max_depth=max_depth, parent=self, parent_side=i)
            self.children[i] = node
            self.neighbors[i] = node
    
    def plot_recursive(self, ax):
        xs, ys = get_poly_xy(self.polygon)
        ax.plot(xs, ys)
        for child in self.children.values():
            child.plot_recursive(ax)

root = Node(name=0, depth=0, max_depth=1)

import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()

root.plot_recursive(ax)

ax.axis('equal')

if False:
    # TODO(lucasw) this isn't working
    drawing = Drawing(2.1, 2.1, origin='center')
    drawing.draw(euclid.Circle(0, 0, 1), fill='#f9f9f9')
    drawing.draw(root.polygon, fill="blue", opacity=1.0)
    print(drawing)
    drawing